<a href="https://colab.research.google.com/github/asepmuhidin/MK-Data-Mining-UPB/blob/main/SVM_GridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/asepmuhidin/MK-Data-Mining-UPB/main/dataset/heart.csv"
df = pd.read_csv(url)
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [22]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['output'])
y = df['output']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((257, 13), (46, 13), (257,), (46,))

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_columns= ['age', 'trtbps', 'chol', 'thalachh','oldpeak']

categorical_columns = ['sex', 'cp','fbs','restecg','exng','slp','caa','thall']


numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessing = ColumnTransformer(
    transformers=[
        ('numeric', numeric_pipeline, numeric_columns),
        ('categorical', categorical_pipeline, categorical_columns)
    ])

In [24]:
from sklearn.svm import SVC
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing),
    ('model', SVC())
])

In [27]:
from sklearn.model_selection import GridSearchCV

parameter={
    "model__C": [0.1, 1, 10, 100, 1000],
    "model__gamma":[1, 0.1, 0.01, 0.001, 0.0001],
    "model__kernel":['rbf']
}
model=GridSearchCV(model_pipeline,param_grid=parameter, cv=5, n_jobs=-1,verbose=1)
model.fit(X_train,y_train)



Fitting 3 folds for each of 25 candidates, totalling 75 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'trtbps',
                                                                          'chol',
                                                                          'thalachh',
                                                                          'oldpeak']),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['sex',
                                                                          'cp',
                                                                          'fbs',
                                                                          'restecg',
                                                                          'exng',
                                                                          'slp',
                                                                          'caa',
                                                                          'thall'])])),
                                       ('model', SVC())]),
             n_jobs=-1,
             param_grid={'model__C': [0.1, 1, 10, 100, 1000],
                         'model__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'model__kernel': ['rbf']},
             verbose=1)

In [28]:
model.score(X_test,y_test)

0.8043478260869565

In [19]:
model.best_params_

{'model__C': 100, 'model__gamma': 0.0001, 'model__kernel': 'rbf'}